In [50]:
import torch
import io
import pandas as pd
from transformers import BertTokenizer

# Force PyTorch to use CPU
#device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)


Using device: cuda


In [56]:


# def load_financial_phrasebank():
#     """Load and preprocess the Financial PhraseBank df."""
#     print("Loading Financial PhraseBank df...")
#     data = []
#     with open("/Sentences_50Agree.txt", "r",
#               encoding="ISO-8859-1") as file:
#         for line in file:
#             text, sentiment = line.rsplit("@", 1)
#             sentiment = sentiment.strip()
#             label = {"neutral": 0, "positive": 1, "negative": 2}.get(sentiment, 0)
#             data.append({"text": text.strip(), "sentiment": label})

#     df = pd.DataFrame(data)
#     return df

def load_financial_phrasebank():
    data = []
    with open("/Sentences_50Agree.txt", "r",
              encoding="ISO-8859-1") as file:
        for line in file:
            text, sentiment = line.rsplit("@", 1)
            sentiment = sentiment.strip()
            # Check what order you used here during training
            label = {"negative": 0, "neutral": 1, "positive": 2}.get(sentiment, 0)
            data.append({"text": text.strip(), "sentiment": label})

    df = pd.DataFrame(data)
    return df


df = load_financial_phrasebank()
df.head()
print(df['sentiment'].value_counts())
# print(df.head())
#
# print(df.loc[df.sentiment == 0].sample(5)[['text', 'sentiment']])


sentiment
1    2879
2    1363
0     604
Name: count, dtype: int64


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [57]:
contents = df.text.values
labels = df.sentiment.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# print(tokenizer.tokenize(contents[0]))
# print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(contents[0])))
# input_ids = torch.tensor(tokenizer.encode('this is a cat', add_special_tokens=True, truncation=True, max_length = 10, padding = True)).unsqueeze(0)
# print(input_ids)

max_len = 0

for sentence in contents:
    input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True))
    max_len = max(max_len, len(input_ids))

max_len

150

In [58]:
input_ids = []
attention_masks = []

for sent in contents:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True, # add [CLS] and [SEP]
                        truncation=True,
                        max_length = 64,           # Pad & truncate all sentences.
                        padding='max_length',
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn masks.
                        return_tensors = 'pt',     # return pytorch tensors.
                   )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

#sentence 0 as list of ids
print('Original: ', contents[0])
print('Token IDs:', input_ids[0])

Original:  According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Token IDs: tensor([  101,  2429,  2000, 12604,  1010,  1996,  2194,  2038,  2053,  3488,
         2000,  2693,  2035,  2537,  2000,  3607,  1010,  2348,  2008,  2003,
         2073,  1996,  2194,  2003,  3652,  1012,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [59]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

# train_size = int(0.85 * len(dataset))
# val_size = len(dataset) - train_size

# # divide dataset by randomly selecting samples.
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))

# # Add these after creating your dataset
# print("Unique labels in dataset:", torch.unique(labels))
# print("Label distribution:", torch.bincount(labels))


# Define split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Compute dataset sizes
train_size = int(train_ratio * len(dataset))
val_size = int(val_ratio * len(dataset))
test_size = len(dataset) - train_size - val_size  # Ensure total size is correct

# Split dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Print dataset sizes
print(f"Train samples: {train_size}")
print(f"Validation samples: {val_size}")
print(f"Test samples: {test_size}")







from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # training samples.
            sampler = RandomSampler(train_dataset), # get batches randomly
            batch_size = batch_size # trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=batch_size
)

Train samples: 3392
Validation samples: 726
Test samples: 728


In [60]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    return_dict = False
)
model.train()
# Tell pytorch to run this model on the GPU.
model.cuda()


params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))



optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [61]:
from transformers import get_linear_schedule_with_warmup

epochs = 2

# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [62]:
# training loop
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


import time
import datetime


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [63]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass.
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)

        loss = outputs[0]
        logits = outputs[1]

        print("\/\/\/\/\/\/\/")
        print("Loss:", loss)
        print("Logits shape:", logits.shape)
        print("Labels shape:", b_labels.shape)
        print("Unique labels:", torch.unique(b_labels))

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 4 ========
Training...
\/\/\/\/\/\/\/
Loss: tensor(1.1559, device='cuda:0', grad_fn=<NllLossBackward0>)
Logits shape: torch.Size([32, 3])
Labels shape: torch.Size([32])
Unique labels: tensor([0, 1, 2], device='cuda:0')
\/\/\/\/\/\/\/
Loss: tensor(1.0919, device='cuda:0', grad_fn=<NllLossBackward0>)
Logits shape: torch.Size([32, 3])
Labels shape: torch.Size([32])
Unique labels: tensor([0, 1, 2], device='cuda:0')
\/\/\/\/\/\/\/
Loss: tensor(1.0407, device='cuda:0', grad_fn=<NllLossBackward0>)
Logits shape: torch.Size([32, 3])
Labels shape: torch.Size([32])
Unique labels: tensor([0, 1, 2], device='cuda:0')
\/\/\/\/\/\/\/
Loss: tensor(1.0041, device='cuda:0', grad_fn=<NllLossBackward0>)
Logits shape: torch.Size([32, 3])
Labels shape: torch.Size([32])
Unique labels: tensor([0, 1, 2], device='cuda:0')
\/\/\/\/\/\/\/
Loss: tensor(1.0151, device='cuda:0', grad_fn=<NllLossBackward0>)
Logits shape: torch.Size([32, 3])
Labels shape: torch.Size([32])
Unique labels: tensor([0, 1

In [64]:
print("\nRunning Test Evaluation...")

t0 = time.time()
model.eval()

total_test_accuracy = 0
total_test_loss = 0

for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        (loss, logits) = model(b_input_ids,
                               token_type_ids=None,
                               attention_mask=b_input_mask,
                               labels=b_labels)

    total_test_loss += loss.item()
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    total_test_accuracy += flat_accuracy(logits, label_ids)

avg_test_accuracy = total_test_accuracy / len(test_dataloader)
avg_test_loss = total_test_loss / len(test_dataloader)

print("  Test Accuracy: {0:.2f}".format(avg_test_accuracy))
print("  Test Loss: {0:.2f}".format(avg_test_loss))
print("  Test Evaluation took: {:}".format(format_time(time.time() - t0)))


Running Test Evaluation...
  Test Accuracy: 0.84
  Test Loss: 0.48
  Test Evaluation took: 0:00:02


In [65]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')

In [66]:
print(os.path.exists("./model_save/"))
print(os.listdir("./"))

True
['.config', 'drive', 'model_save', 'sample_data']


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
# Copy the model files to a directory in your Google Drive.
#!cp -r ./model_save/ "./drive/Shared drives/ChrisMcCormick.AI/Blog Posts/BERT Fine-Tuning/"


import os

# Define the correct path for the models_saved folder
save_path = "/content/drive/MyDrive/models_saved/"

# Create the folder if it doesn't exist
os.makedirs(save_path, exist_ok=True)
model.save_pretrained(save_path)

In [89]:
# from sklearn.utils import class_weight
# import torch
# import numpy as np
# from transformers import BertForSequenceClassification, BertTokenizer

# # Load the model and tokenizer
# model_path = "/content/drive/MyDrive/models_saved"
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertForSequenceClassification.from_pretrained(model_path)
# model.eval()

# # Ensure labels are loaded correctly
# if isinstance(labels, np.ndarray):  # If labels are a NumPy array
#     labels_numpy = labels.flatten()  # Ensure it's a 1D array
# else:
#     raise ValueError(f"Error: labels is of type {type(labels)} and is not a NumPy array.")

# # Check the unique values in the labels array
# print("Unique labels before cleaning:", np.unique(labels_numpy))

# # Check if any ellipsis or invalid values are present
# if np.any(labels_numpy == Ellipsis):
#     print("Warning: Ellipsis ('...') detected in labels array.")
#     # Clean up: remove ellipsis values if present
#     labels_numpy = labels_numpy[labels_numpy != Ellipsis]

# # Ensure labels are clean and of integer type (if necessary)
# labels_numpy = labels_numpy.astype(int)

# # Check the first few labels to confirm the data
# print("Sample labels:", labels_numpy[:10])

# # Compute class weights using the labels
# try:
#     class_weights = class_weight.compute_class_weight(
#         class_weight='balanced',
#         classes=np.unique(labels_numpy),  # Classes should be unique labels
#         y=labels_numpy  # Your labels
#     )
#     print("Class weights computed:", class_weights)
# except ValueError as e:
#     print("Error computing class weights:", e)
#     print("Unique values in labels:", np.unique(labels_numpy))

# # Convert class weights to a tensor and move to device (GPU if available)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# class_weights_tensor = torch.from_numpy(class_weights).float().to(device)

# # Print the results
# print("Class weights tensor:", class_weights_tensor)


TypeError: '<' not supported between instances of 'ellipsis' and 'int'

In [96]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Path to the saved model
model_path = "/content/drive/MyDrive/models_saved"  # Adjust this path if needed

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load the model
model = BertForSequenceClassification.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Define a function to preprocess the input text
def preprocess_text(text, tokenizer, max_length=512):
    # Tokenize the input text
    encoding = tokenizer.encode_plus(
        text,                      # Input text
        add_special_tokens=True,    # Add special tokens like [CLS] and [SEP]
        max_length=max_length,     # Maximum length of the tokenized sequence
        padding='max_length',       # Pad to max_length
        truncation=True,            # Truncate if it's longer than max_length
        return_tensors='pt',        # Return as PyTorch tensor
    )

    return encoding

# Example input text (news article)
# text = "The stock market saw a significant drop today after disappointing earnings reports from major tech companies."
text = "The stock market saw a significant rise today after announced new product launch"


# Preprocess the input text
inputs = preprocess_text(text, tokenizer)

# Make the prediction
with torch.no_grad():  # Disable gradient calculations for inference
    outputs = model(**inputs)

# Get the logits (the raw prediction scores)
logits = outputs[0]

# Convert logits to probabilities using softmax
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# Get the predicted sentiment (class)
predicted_class = torch.argmax(probabilities, dim=-1).item()

# Print the result
print(f"Predicted class: {predicted_class}")
print(f"Probabilities: {probabilities}")

sentiment_classes = ["negative", "neutral", "positive"]

print(f"Predicted sentiment: {sentiment_classes[predicted_class]}")


Predicted class: 2
Probabilities: tensor([[0.0032, 0.0064, 0.9904]])
Predicted sentiment: positive
